In [ ]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# Common imports
import numpy as np
import os
import pandas as pd

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2

# plt.style.use(['seaborn-pastel','dark_background'])
mpl.rcParams['figure.facecolor'] = 'whitesmoke'
mpl.rcParams['axes.facecolor'] = 'white'
# mpl.rcParams['figure.figsize'] = 7, 10
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# show multiple outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# download images
import urllib.request

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "fingerprint"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

def download_fig(fig_id, fig_url, fig_extension="jpg"):
    print("Downloading", fig_url)
    req = urllib.request.Request(url)
    # Customize the default User-Agent header value:
    req.add_header('User-Agent', 'Mozilla/5.0')
    response = urllib.request.urlopen(req)
    image = response.read()
    filename = fig_id + "." + fig_extension
    with open(os.path.join(IMAGES_PATH, filename), "wb") as file:
        file.write(image)

# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")


print("Setup finished!")

In [ ]:
def process(file_name, sorted_mac):
    filef = open(file_name, "r")
    line = filef.readline()
    data_dict = {'grid_id': [], 'x': [], 'y': [], 'mac': [], 'type': [], 'rssi': []}
    grid_id = 0
    while True:
            if line.strip() is not '':
                grid_id += 1
                grid_title = line.split(" ")
                print(grid_title)
                grid_pos = [float(grid_title[0]), float(grid_title[1])]
                for i in range(int(grid_title[2])):
                    line = filef.readline().rstrip("\n")
                    record = line.split("|")
                    ap = record[0].split(" ")
                    data_dict['grid_id'].append(grid_id)
                    data_dict['x'].append(grid_pos[0])
                    data_dict['y'].append(grid_pos[1])
                    data_dict['mac'].append(ap[0][0:17])
                    data_dict['type'].append(ap[2])
                    rssi_list = record[1].strip().split(' ') if record[1].strip() != '' else [ap[1]]
                    data_dict['rssi'].append(float(np.array(rssi_list).astype(np.float).mean()))
            line = filef.readline()
            if not line:
                break
    filef.close()
    data_frame = pd.DataFrame.from_dict(data_dict)
    
    grouped = data_frame.groupby(by=['grid_id'])
    grid_sum = len(grouped.groups)
    theta = np.full(shape=(grid_sum, len( sorted_mac)), fill_value=-100.)
    grid_index = 0
    grid_position = np.full(shape=(grid_sum, 2), fill_value=0)
    for name, group in grouped:
            for index, row in group.iterrows():
                if row['mac'] in sorted_mac:
                    ap_index = sorted_mac.index(row['mac'])
                    theta[grid_index, ap_index] = row['rssi']
            grid_position[grid_index] = [group["x"].mean(), group['y'].mean()]
            grid_index += 1 
    data = {"theta": theta, "position": grid_position}
    df = pd.DataFrame({k:list(v) for k,v in data.items()})
    return df

In [ ]:
# grouped_df = data_frame.groupby(by=['x','y'])
# # for key,item in grouped_df:
# #     a_group = grouped_df.get_group(key)
# #     print(a_group, "\n")
# sorted_mac = None
# filter_mac = None
# use_type = 'w'
# grid_sum = len(grouped_df.groups)
# if not sorted_mac:
#     mac = set(data_frame[data_frame['type'] == use_type]['mac'].unique())
#     sorted_mac = sorted(mac & filter_mac if filter_mac else mac)
# sorted_mac

In [ ]:
# Download the California image
download_fig(url, "minespark")


In [ ]:
def plotScatter(points, fig_id):
    

In [ ]:


tmpdata = data_frame.copy()
tmpdata["y"] += 0.25
housing_img=mpimg.imread(os.path.join(IMAGES_PATH, filename))
# housing_img = cv2.flip(housing_img, 0)
ax = tmpdata.plot(kind="scatter", x="x", y="y", alpha=0.5,
    figsize=(10,7),
    sharex=False)
ax=plt.gca()                            # get the axis
ax.set_ylim(ax.get_ylim()[::-1])        # invert the axis
ax.xaxis.tick_top()                     # and move the X-Axis      
ax.yaxis.tick_left()                    # remove right y-Ticks
plt.imshow(housing_img, extent=[0, 10, 0, 7],alpha=0.5,
           cmap=plt.get_cmap("jet"))

# save_fig(filename+"-test")
plt.show()

In [ ]:
grouped = data_frame.groupby(by=['grid_id'])
grid_sum = len(grouped.groups)

In [ ]:
theta = np.full(shape=(grid_sum, len( sorted_mac)), fill_value=-100.)
grid_index = 0
grid_position = np.full(shape=(grid_sum, 2), fill_value=0)
for name, group in grouped:
            for index, row in group.iterrows():
                if row['mac'] in sorted_mac:
                    ap_index = sorted_mac.index(row['mac'])
                    theta[grid_index, ap_index] = row['rssi']
            grid_position[grid_index] = [group["x"].mean(), group['y'].mean()]
            grid_index += 1

In [ ]:
data = {"theta": theta, "position": grid_position}
data

In [ ]:
df = pd.DataFrame({k:list(v) for k,v in data.items()})

In [ ]:
df

In [ ]:
df["theta"][16]

In [ ]:
def plot(target, predit):
    housing_img=mpimg.imread(os.path.join(IMAGES_PATH, filename))
    housing_img = cv2.flip(housing_img, 0)
    plt.scatter(target[:, 0], target[:, 1], c='b', marker='s', label='-1')
    for i in range(target.shape[0]):
        plt.text(target[i, 0], target[i, 1], str(i))

    plt.scatter(predict[:, 0], predict[:, 1], c='r', marker='s', label='-1')
    for i in range(predict.shape[0]):
        plt.text(predict[i,0], predict[i,1], str(i))

    plt.imshow(housing_img, extent=[0, 10, 0, 7],alpha=0.5,
            cmap=plt.get_cmap("jet"))

In [ ]:
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()


In [ ]:
from sklearn.ensemble import RandomForestRegressor
forest_reg = RandomForestRegressor(n_estimators=100, random_state=42)

In [ ]:
from sklearn.svm import SVR
svm_reg = SVR(kernel="linear")

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()

In [ ]:
from sklearn.model_selection import cross_val_predict, cross_val_score

predict = cross_val_predict(knn, data["theta"], data["position"],
                         method="predict", cv=3)
                        
# predict

In [ ]:
target = data["position"]
plot(target, predict)

In [ ]:
# {'target': target, 'predict': predict}

In [ ]:
score = cross_val_score(knn, data["theta"], data["position"],
                         scoring="neg_root_mean_squared_error", cv=3)

In [ ]:
score

In [ ]:
test_df = process('test11_16.txt', sort_max)

In [ ]:
test_df

In [ ]:
final_predit = knn.predict(test_df["theta"])